# Week 11: 스마트 CCTV 모니터링 시스템

## 🎓 교육용 완전 구현

**목표**:
- YOLOv8 객체 탐지
- ByteTrack 추적
- ROI 침입/배회 감지
- 히트맵 분석
- 통합 시스템

**실행 환경**: Google Colab (무료 GPU)

---

## Section 1: 환경 설정

In [ ]:
# 패키지 설치
!pip install ultralytics opencv-python-headless matplotlib numpy

import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
from collections import deque, defaultdict
from google.colab.patches import cv2_imshow
import time

print("✅ 환경 설정 완료")

In [ ]:
# 샘플 영상 다운로드 (또는 업로드)
from google.colab import files

print("샘플 영상을 업로드하거나, 웹캠 테스트용 이미지 생성")
# uploaded = files.upload()  # 파일 업로드

# 테스트 이미지 생성
test_frame = np.ones((480, 640, 3), dtype=np.uint8) * 200
cv2.putText(test_frame, "Smart CCTV Test Frame", (150, 240),
           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

plt.imshow(cv2.cvtColor(test_frame, cv2.COLOR_BGR2RGB))
plt.title("Test Frame")
plt.show()

print("✅ 샘플 데이터 준비 완료")

## Section 2: YOLOv8 객체 탐지

In [ ]:
# YOLOv8 모델 로드
model = YOLO('yolov8n.pt')  # nano 모델 (자동 다운로드)
print("✅ YOLOv8 모델 로드 완료")

# 추론 테스트
results = model(test_frame, conf=0.5, verbose=False)

# 결과 시각화
result_img = results[0].plot()
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
plt.title("YOLOv8 Detection Result")
plt.axis('off')
plt.show()

# 탐지 통계
num_detections = len(results[0].boxes)
print(f"탐지된 객체 수: {num_detections}")

## Section 3: ByteTrack 추적 (간소화)

In [ ]:
# SimpleTrack 클래스
class SimpleTrack:
    def __init__(self, track_id, bbox, class_id, confidence):
        self.id = track_id
        self.bbox = bbox
        self.class_id = class_id
        self.confidence = confidence
        self.history = deque(maxlen=30)
        self.history.append(self.get_center())
        self.age = 1
        self.time_since_update = 0

    def get_center(self):
        x1, y1, x2, y2 = self.bbox
        return (int((x1 + x2) / 2), int((y1 + y2) / 2))

    def update(self, bbox, confidence):
        self.bbox = bbox
        self.confidence = confidence
        self.age += 1
        self.time_since_update = 0
        self.history.append(self.get_center())

print("✅ SimpleTrack 클래스 정의 완료")

In [ ]:
# SimpleByteTracker 클래스
class SimpleByteTracker:
    def __init__(self, high_thresh=0.5, low_thresh=0.1, max_age=30):
        self.high_thresh = high_thresh
        self.low_thresh = low_thresh
        self.max_age = max_age
        self.tracks = {}
        self.next_id = 1

    def update(self, detections):
        # 간소화: 고신뢰도만 사용
        high_dets = [d for d in detections if d['conf'] >= self.high_thresh]

        # Greedy matching
        matched = set()
        for det in high_dets:
            best_iou, best_tid = 0, None
            for tid, track in self.tracks.items():
                iou = self._calculate_iou(det['bbox'], track.bbox)
                if iou > best_iou and iou >= 0.3:
                    best_iou, best_tid = iou, tid

            if best_tid:
                self.tracks[best_tid].update(det['bbox'], det['conf'])
                matched.add(best_tid)
            else:
                # 새 Track
                self.tracks[self.next_id] = SimpleTrack(
                    self.next_id, det['bbox'], det['class'], det['conf']
                )
                self.next_id += 1

        # 오래된 Track 제거
        to_remove = []
        for tid, track in self.tracks.items():
            if tid not in matched:
                track.time_since_update += 1
                if track.time_since_update > self.max_age:
                    to_remove.append(tid)

        for tid in to_remove:
            del self.tracks[tid]

        return self.tracks

    def _calculate_iou(self, box1, box2):
        x1_min, y1_min, x1_max, y1_max = box1
        x2_min, y2_min, x2_max, y2_max = box2

        inter_x_min, inter_y_min = max(x1_min, x2_min), max(y1_min, y2_min)
        inter_x_max, inter_y_max = min(x1_max, x2_max), min(y1_max, y2_max)

        inter_area = max(0, inter_x_max - inter_x_min) * max(0, inter_y_max - inter_y_min)
        area1 = (x1_max - x1_min) * (y1_max - y1_min)
        area2 = (x2_max - x2_min) * (y2_max - y2_min)
        union_area = area1 + area2 - inter_area

        return inter_area / union_area if union_area > 0 else 0

tracker = SimpleByteTracker()
print("✅ SimpleByteTracker 클래스 정의 완료")

## Section 4: ROI 침입 감지

In [ ]:
# ROI 설정 (예제)
roi_polygon = np.array([
    [100, 200],
    [500, 200],
    [500, 400],
    [100, 400]
], dtype=np.int32)

# ROI 시각화
roi_frame = test_frame.copy()
overlay = roi_frame.copy()
cv2.fillPoly(overlay, [roi_polygon], (0, 0, 255))
roi_frame = cv2.addWeighted(roi_frame, 0.7, overlay, 0.3, 0)
cv2.polylines(roi_frame, [roi_polygon], True, (0, 0, 255), 3)

plt.figure(figsize=(10, 6))
plt.imshow(cv2.cvtColor(roi_frame, cv2.COLOR_BGR2RGB))
plt.title("ROI Setup")
plt.axis('off')
plt.show()

print("✅ ROI 설정 완료")

In [ ]:
# 침입 감지 함수
def check_intrusion(tracks, roi_polygon, intrusion_records, threshold_seconds=3, fps=30):
    alerts = []
    current_time = time.time()

    for tid, track in tracks.items():
        center = track.get_center()
        is_inside = cv2.pointPolygonTest(roi_polygon, center, False) >= 0

        if is_inside:
            if tid not in intrusion_records:
                intrusion_records[tid] = current_time

            duration = current_time - intrusion_records[tid]
            if duration >= threshold_seconds:
                alerts.append({
                    'type': 'INTRUSION',
                    'track_id': tid,
                    'duration': duration,
                    'position': center
                })
        else:
            if tid in intrusion_records:
                del intrusion_records[tid]

    return alerts

print("✅ 침입 감지 함수 정의 완료")

## Section 5: 배회 감지

In [ ]:
# 배회 감지 함수
def check_loitering(tracks, min_duration_seconds=10, max_movement_pixels=100, fps=30):
    alerts = []
    min_frames = int(min_duration_seconds * fps)

    for tid, track in tracks.items():
        if len(track.history) < min_frames:
            continue

        recent_history = list(track.history)[-min_frames:]
        total_movement = sum(
            np.sqrt((recent_history[i][0] - recent_history[i-1][0])**2 +
                   (recent_history[i][1] - recent_history[i-1][1])**2)
            for i in range(1, len(recent_history))
        )

        if total_movement < max_movement_pixels:
            alerts.append({
                'type': 'LOITERING',
                'track_id': tid,
                'movement': total_movement,
                'position': track.get_center()
            })

    return alerts

print("✅ 배회 감지 함수 정의 완료")

## Section 6: 히트맵 생성

In [ ]:
# 히트맵 클래스
class HeatmapGenerator:
    def __init__(self, frame_shape, decay_factor=0.995):
        self.height, self.width = frame_shape
        self.heatmap = np.zeros((self.height, self.width), dtype=np.float32)
        self.decay_factor = decay_factor

    def update(self, tracks):
        self.heatmap *= self.decay_factor

        for track in tracks.values():
            center = track.get_center()
            x, y = center

            if 0 <= x < self.width and 0 <= y < self.height:
                # Gaussian blob
                radius = 20
                for i in range(max(0, y-radius), min(self.height, y+radius)):
                    for j in range(max(0, x-radius), min(self.width, x+radius)):
                        dist = np.sqrt((j - x)**2 + (i - y)**2)
                        if dist <= radius:
                            value = np.exp(-(dist**2) / (2 * (radius/3)**2))
                            self.heatmap[i, j] += value

    def get_overlay(self, frame, alpha=0.5):
        heatmap_norm = cv2.normalize(self.heatmap, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        heatmap_colored = cv2.applyColorMap(heatmap_norm, cv2.COLORMAP_JET)
        return cv2.addWeighted(frame, 1-alpha, heatmap_colored, alpha, 0)

heatmap_gen = HeatmapGenerator((480, 640))
print("✅ 히트맵 생성기 준비 완료")

## Section 7: 통합 시스템

In [ ]:
# 통합 실행 (샘플)
print("=== 통합 스마트 CCTV 시스템 실행 ===")

intrusion_records = {}
event_log = []

# 샘플 프레임 처리
for i in range(10):
    # YOLOv8 탐지
    results = model(test_frame, conf=0.3, classes=[0, 2], verbose=False)

    detections = []
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            detections.append({
                'bbox': [x1, y1, x2, y2],
                'conf': float(box.conf[0]),
                'class': int(box.cls[0])
            })

    # ByteTrack 추적
    tracks = tracker.update(detections)

    # 침입/배회 감지
    intrusion_alerts = check_intrusion(tracks, roi_polygon, intrusion_records)
    loitering_alerts = check_loitering(tracks)

    all_alerts = intrusion_alerts + loitering_alerts
    event_log.extend(all_alerts)

    # 히트맵 업데이트
    heatmap_gen.update(tracks)

    if all_alerts:
        for alert in all_alerts:
            print(f"[Frame {i}] 🚨 {alert['type']}: Track {alert['track_id']}")

# 최종 결과 시각화
final_frame = test_frame.copy()
heatmap_overlay = heatmap_gen.get_overlay(final_frame, alpha=0.5)

plt.figure(figsize=(14, 8))
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(final_frame, cv2.COLOR_BGR2RGB))
plt.title("Original Frame")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(heatmap_overlay, cv2.COLOR_BGR2RGB))
plt.title("Heatmap Overlay")
plt.axis('off')

plt.tight_layout()
plt.show()

print(f"\n✅ 통합 시스템 실행 완료")
print(f"총 이벤트 수: {len(event_log)}")

## Section 8: 통계 및 결과

### 최종 요약

```
✅ YOLOv8 탐지: 완료
✅ ByteTrack 추적: 완료
✅ ROI 침입 감지: 완료
✅ 배회 감지: 완료
✅ 히트맵 분석: 완료
```

### 다음 단계

1. **실제 영상 테스트**: 자신의 CCTV 영상으로 테스트
2. **성능 최적화**: TensorRT, 프레임 스킵
3. **기능 확장**: 얼굴 블러, 차량 번호판 인식
4. **배포**: Streamlit 앱, Docker 컨테이너

---

**교육 목적으로 구현되었습니다. 실제 사용 시 개인정보 보호법 준수하세요!**

🚀 **Happy Coding!**